### https://github.com/krishnaik06/Fake-New-LSTM

### Dataset: https://www.kaggle.com/c/fake-news/data#

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/home/vinoth/Learning/Python/Workouts/fake-news/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df=df.dropna()

In [6]:
X = df.drop('label', axis=1)
X[:5]

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [7]:
Y = df['label']
Y[:5]

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [12]:
Y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [13]:
X.shape

(18285, 4)

In [14]:
Y.shape

(18285,)

In [15]:
import tensorflow as tf
print(tf.__version__)

2.4.0


In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [17]:
# Vocabulary Size - Consider this as dictionary size
voc_size = 5000

## OneHot Representation

In [18]:
messages = X.copy()
print(messages)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [19]:
messages.reset_index(inplace=True)

In [20]:
messages['title']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
18280    Rapper T.I.: Trump a ’Poster Child For White S...
18281    N.F.L. Playoffs: Schedule, Matchups and Odds -...
18282    Macy’s Is Said to Receive Takeover Approach by...
18283    NATO, Russia To Hold Parallel Exercises In Bal...
18284                            What Keeps the F-35 Alive
Name: title, Length: 18285, dtype: object

### Data Preprocessing

In [21]:
import nltk as nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/vinoth/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [25]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[2776, 167, 3357, 266, 260, 600, 1816, 16, 4418, 20],
 [4235, 1741, 1551, 4211, 1285, 1880, 2457],
 [3210, 3246, 2365, 2153],
 [2574, 172, 4659, 4404, 987, 3593],
 [3263, 1285, 2578, 1607, 1755, 1116, 1285, 2266, 4402, 3533],
 [570,
  4013,
  4617,
  4711,
  932,
  2956,
  3032,
  1867,
  1784,
  3137,
  3765,
  4180,
  792,
  3426,
  2457],
 [2223, 4603, 126, 4181, 1619, 35, 4548, 1006, 2093, 3689, 3820],
 [2522, 3879, 3002, 1306, 4047, 4787, 2956, 52, 2093, 3689, 3820],
 [3261, 1896, 140, 3915, 102, 1267, 1202, 2627, 2956, 2344],
 [4430, 4492, 3107, 906, 4253, 4222, 667, 2600],
 [1821, 4134, 1364, 1214, 4480, 4362, 3717, 3122, 3407, 2375, 2590],
 [4404, 2796, 260, 1267, 2956, 4047],
 [4964, 4910, 4194, 2556, 1722, 750, 714, 56, 1451],
 [1167, 4243, 1725, 3730, 2772, 2694, 3456, 2093, 3689, 3820],
 [1808, 2148, 4103, 2099, 3053, 2093, 3689, 3820],
 [499, 2815, 3326, 3919, 4496, 3349, 4264, 681, 4330, 4068],
 [4422, 3244, 1741],
 [4491, 2862, 1501, 3703, 2956, 1901, 1788, 2457],
 [246

### Apply padding to set same length for all sentence since word embedding requires it

In [27]:
# maxlen - Optional Int, maximum length of all sequences. 
#If not provided, sequences will be padded to the length of the longest individual sequence.
# https://stackoverflow.com/questions/52086530/why-is-maxlen-set-to-256-in-tensorflow-text-classification-tutorial
embedded_docs = pad_sequences(onehot_repr,padding='pre')
print(embedded_docs.shape)
print(embedded_docs.shape[1])
print(embedded_docs)

(18285, 47)
47
[[   0    0    0 ...   16 4418   20]
 [   0    0    0 ... 1285 1880 2457]
 [   0    0    0 ... 3246 2365 2153]
 ...
 [   0    0    0 ... 2093 3689 3820]
 [   0    0    0 ... 3893  813 4743]
 [   0    0    0 ...  558 2572 1414]]


### Creating Model

In [28]:
embedding_vector_feature = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_length=embedded_docs.shape[1]))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 47, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
print(embedded_docs.shape,Y.shape)
print(len(embedded_docs),len(Y))
print(type(embedded_docs), type(Y))

(18285, 47) (18285,)
18285 18285
<class 'numpy.ndarray'> <class 'pandas.core.series.Series'>


In [30]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)
print(type(X_final), type(Y_final))
print(embedded_docs.shape,Y.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(18285, 47) (18285,)


In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,Y_final,test_size=0.2, random_state=42)

## Model Training

In [32]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10, batch_size=64)

Epoch 1/10
229/229 [==============================] - 44s 158ms/step - loss: 0.4609 - accuracy: 0.7626 - val_loss: 0.2017 - val_accuracy: 0.9103
Epoch 2/10
229/229 [==============================] - 23s 100ms/step - loss: 0.1379 - accuracy: 0.9487 - val_loss: 0.2062 - val_accuracy: 0.9155
Epoch 3/10
229/229 [==============================] - 20s 86ms/step - loss: 0.0846 - accuracy: 0.9714 - val_loss: 0.2316 - val_accuracy: 0.9089
Epoch 4/10
229/229 [==============================] - 20s 86ms/step - loss: 0.0617 - accuracy: 0.9798 - val_loss: 0.2935 - val_accuracy: 0.9130
Epoch 5/10
229/229 [==============================] - 21s 91ms/step - loss: 0.0405 - accuracy: 0.9864 - val_loss: 0.3118 - val_accuracy: 0.9078
Epoch 6/10
229/229 [==============================] - 20s 86ms/step - loss: 0.0229 - accuracy: 0.9932 - val_loss: 0.3660 - val_accuracy: 0.9089
Epoch 7/10
229/229 [==============================] - 20s 86ms/step - loss: 0.0145 - accuracy: 0.9959 - val_loss: 0.4105 - val_accurac

## Prediction Metrics

In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
def prediction_metrics(y_pred, y_test):
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print('Accuracy score : ', accuracy_score(y_test, y_pred))
    print('Classification Report\n',classification_report(y_test, y_pred))

In [34]:
print(model.predict(X_test))
y_pred = model.predict_classes(X_test)
print(y_pred)
prediction_metrics(y_pred,y_test)

[[9.9993467e-01]
 [4.9380800e-05]
 [3.7989020e-04]
 ...
 [9.9971104e-01]
 [2.1162073e-07]
 [9.9996090e-01]]


/home/vinoth/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[1]
 [0]
 [0]
 ...
 [1]
 [0]
 [1]]
[[1855  227]
 [ 100 1475]]
Accuracy score :  0.9105824446267432
Classification Report
               precision    recall  f1-score   support

           0       0.95      0.89      0.92      2082
           1       0.87      0.94      0.90      1575

    accuracy                           0.91      3657
   macro avg       0.91      0.91      0.91      3657
weighted avg       0.91      0.91      0.91      3657



## Adding dropout layer

In [35]:
from tensorflow.keras.layers import Dropout
embedding_vector_feature = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_length=embedded_docs.shape[1]))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 47, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 47, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
y_pred1 = model.predict_classes(X_test)
print(y_pred1)
prediction_metrics(y_pred1,y_test)

[[1]
 [0]
 [0]
 ...
 [0]
 [0]
 [1]]
[[1591  491]
 [ 570 1005]]
Accuracy score :  0.7098714793546623
Classification Report
               precision    recall  f1-score   support

           0       0.74      0.76      0.75      2082
           1       0.67      0.64      0.65      1575

    accuracy                           0.71      3657
   macro avg       0.70      0.70      0.70      3657
weighted avg       0.71      0.71      0.71      3657

